In [ ]:
import os
import re
import pandas as pd
from fastapi import FastAPI, UploadFile, File, HTTPException, BackgroundTasks
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import List, Optional
import pymongo
from detoxify import Detoxify
import uvicorn
from bson import json_util
import json

app = FastAPI(title="Facebook Comments Toxicity Analyzer")

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["facebook_comments"]
collection = db["toxicity_analysis"]

In [ ]:
model = Detoxify('original')

In [ ]:
def clean_text(text):
    if text == "No text found":
        return ""
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'…\s*En voir plus', '', text)
    return text

def analyze_toxicity(text):
    if not text or text == "":
        return {
            "toxicity": 0.0,
            "severe_toxicity": 0.0,
            "obscene": 0.0,
            "threat": 0.0,
            "insult": 0.0,
            "identity_attack": 0.0
        }
    results = model.predict(text)
    return {k: float(v) for k, v in results.items()}

In [ ]:
def process_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        results = []
        for _, row in df.iterrows():
            if row['text'] == "No text found":
                continue
            cleaned_text = clean_text(row['text'])
            toxicity_scores = analyze_toxicity(cleaned_text)
            result = {
                "author": row['author'],
                "date": row['date'],
                "text": row['text'],
                "url": row['url'],
                "cleaned_text": cleaned_text,
                **toxicity_scores
            }
            results.append(result)
            collection.insert_one(result)
        results_df = pd.DataFrame(results)
        results_df.to_csv(file_path.replace('.csv', '_analyzed.csv'), index=False)
        return len(results)
    except Exception as e:
        print(f"Error processing CSV: {str(e)}")
        raise e

In [ ]:
text = "Tu es vraiment bête et inutile. Va-t'en."
cleaned = clean_text(text)
toxicity_scores = analyze_toxicity(cleaned)

print("Texte nettoyé:", cleaned)
print("Scores de toxicité:", toxicity_scores)

### ▶️ Pour lancer l'API en local :

Exécuter cette commande dans un terminal (pas ici dans Jupyter) :

```
uvicorn app:app --reload
```